In [1]:
from kafka import  KafkaProducer
import sys, os, json
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext
import os
import sys
from pyspark.sql.functions import *
from pyspark.sql.functions import to_json, struct


In [2]:
from confluent_kafka import Producer

In [ ]:
#pip install confluent-kafka

In [3]:
topic_name = 'quickstart-events4'
bootstrap_server="localhost:9092"

In [4]:
scala_version = '2.12'  # TODO: Ensure this is correct
spark_version = '3.3.0'

packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.5.1'
]

args = os.environ.get('PYSPARK_SUBMIT_ARGS', '')
if not args:
    args = f'--packages {",".join(packages)}'
    print('Using packages', packages)
    os.environ['PYSPARK_SUBMIT_ARGS'] = f'{args} pyspark-shell'
else:
    print(f'Found existing args: {args}')

Using packages ['org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0', 'org.apache.kafka:kafka-clients:3.5.1']


In [5]:
# set Java home
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk1.8.0_251"

In [6]:
conf = SparkConf() \
    .setAppName("ETLPipeline") \
    .setMaster("local") \
    .set("spark.driver.extraClassPath","c:/pyspark/*")
sc = SparkContext.getOrCreate(conf=conf)
etl = SparkSession(sc)

In [7]:
uid="chaithanya"
pwd="12345678"

#sql db details
server = "localhost"
src_db = "prac"
src_driver = "com.microsoft.sqlserver.jdbc.SQLServerDriver"

In [8]:
src_url = f"jdbc:sqlserver://{server}:1433;databaseName={src_db};user={uid};password={pwd};"

In [9]:
query="""select emp_id,emp_name,salary,manager_id from prac.dbo.emp_manager"""
table="emp_manager"

In [10]:
dfs=etl.read. \
    format("jdbc"). \
    options(driver=src_driver, user=uid, password=pwd, url=src_url,query=query). \
    load()
dfs.show()

+------+--------+------+----------+
|emp_id|emp_name|salary|manager_id|
+------+--------+------+----------+
|     1|   Ankit| 10000|         4|
|     2|   Mohit| 15000|         5|
|     3|   Vikas| 10000|         4|
|     4|   Rohit|  5000|         2|
|     5|   Mudit| 12000|         6|
|     6|    Agam| 12000|         2|
|     7|  Sanjay|  9000|         2|
|     8|  Ashish|  5000|         2|
|     9|  likhit|  1600|         4|
+------+--------+------+----------+



In [11]:
dfs.show()

+------+--------+------+----------+
|emp_id|emp_name|salary|manager_id|
+------+--------+------+----------+
|     1|   Ankit| 10000|         4|
|     2|   Mohit| 15000|         5|
|     3|   Vikas| 10000|         4|
|     4|   Rohit|  5000|         2|
|     5|   Mudit| 12000|         6|
|     6|    Agam| 12000|         2|
|     7|  Sanjay|  9000|         2|
|     8|  Ashish|  5000|         2|
|     9|  likhit|  1600|         4|
+------+--------+------+----------+



In [ ]:
'''kafka_df = dfs.selectExpr("CAST(emp_id AS STRING) AS key", "to_json(struct(*)) AS value")
kafka_df.show()
'''



In [ ]:
'''kafka_df \
    .write \
    .format("kafka") \
    .option("kafka.bootstrap.servers", bootstrap_server) \
    .option("topic", topic_name) \
    .save()
print("Data written to Kafka topic 'Start'")
'''

In [12]:
from confluent_kafka import Producer

In [31]:
def transform_data_for_kafka(dfs, key_column):
    kafka_df = dfs.selectExpr(f"CAST({key_column} AS STRING) AS key", f"to_json(struct(*)) AS value").collect()
    return kafka_df

In [32]:
a = transform_data_for_kafka(dfs,'emp_id')
print(a)

[Row(key='1', value='{"emp_id":1,"emp_name":"Ankit","salary":10000,"manager_id":4}'), Row(key='2', value='{"emp_id":2,"emp_name":"Mohit","salary":15000,"manager_id":5}'), Row(key='3', value='{"emp_id":3,"emp_name":"Vikas","salary":10000,"manager_id":4}'), Row(key='4', value='{"emp_id":4,"emp_name":"Rohit","salary":5000,"manager_id":2}'), Row(key='5', value='{"emp_id":5,"emp_name":"Mudit","salary":12000,"manager_id":6}'), Row(key='6', value='{"emp_id":6,"emp_name":"Agam","salary":12000,"manager_id":2}'), Row(key='7', value='{"emp_id":7,"emp_name":"Sanjay","salary":9000,"manager_id":2}'), Row(key='8', value='{"emp_id":8,"emp_name":"Ashish","salary":5000,"manager_id":2}'), Row(key='9', value='{"emp_id":9,"emp_name":"likhit","salary":1600,"manager_id":4}')]


In [33]:
def publish_message(producer_instance, topic_name, key, value):
    try:
        producer_instance.produce(topic_name, key=key, value=value)
        producer_instance.flush()
    except Exception as ex:
        print('Exception in publishing message:')
        print(str(ex))

In [34]:
def publish_to_kafka(dfs, kafka_bootstrap_servers, topic, key_column):
    producer_config = {
        'bootstrap.servers': kafka_bootstrap_servers,
        'client.id': 'kafka-producer'
    }

    producer = Producer(producer_config)

    kafka_df = transform_data_for_kafka(dfs, key_column)

    for row in kafka_df:
        key = row.key
        value = row.value
        publish_message(producer, topic, key, value)

In [35]:
publish_to_kafka(dfs, "localhost:9092", 'quickstart-events4','emp_id')
